In [192]:
library(dplyr)

In [193]:
general_dfs = list()
for (i in 1:10) {
    general_dfs[[i]] = read.csv(paste('../results/general_permutation_tests', as.character(i), '.csv', sep = ''))
}

In [194]:
residuals_dfs = list()
for (i in 1:10) {
    residuals_df[[i]] = read.csv(paste('../results/residual_permutation_tests', as.character(i), '.csv', sep = ''))
}

In [195]:
ergm_df = read.csv('../results/ergm_permutation_tests.csv')
gravity_results = read.csv('../results/gravity_model_results.csv')

In [196]:
colnames(gravity_results) = c('effect', 'p_value')

In [197]:
all_models = list()
all_models[['general_models']] = general_dfs
all_models[['residual_models']] = residuals_df
all_models[['ergm_model']] = ergm_df

In [198]:
check_significance = function(p_values) {
    significance = matrix(' ', nrow = length(p_values))
    significance[which(p_values < .1)] = '.'
    significance[which(p_values < .05)] = '*'
    significance[which(p_values < .01)] = '**'
    significance[which(p_values < .001)] = '***'
    return(significance)
}

In [199]:
compute_errors = function(results, correct_results) {
    false_positives = 0
    false_negatives = 0
    merged_df = merge(results, correct_results, by = 'effect')
    false_positives = sum((merged_df$sign_level.x != ' ') & (merged_df$sign_level.y == ' '))
    false_negatives = sum((merged_df$sign_level.x == ' ') & (merged_df$sign_level.y != ' '))
    return(c(false_positives, false_negatives))
}

In [200]:
for (model in c('general_models', 'residual_models')) {
    for (i in 1:length(all_models[[model]])) {
        all_models[[model]][[i]][, 'sign_level'] = check_significance(all_models[[model]][[i]][, 'p_value'])
    }
}

In [201]:
all_models[['ergm_model']][, 'sign_level'] = check_significance(all_models[['ergm_model']][, 'p_value'])
gravity_results[, 'sign_level'] = check_significance(gravity_results$p_value)

In [202]:
results = data.frame()
for (model in c('general_models', 'residual_models')) {
    for (i in 1:length(all_models[[model]])) {
        errors = compute_errors(results = all_models[[model]][[i]], correct_results = gravity_results)
        results = results %>% 
            dplyr::bind_rows(data.frame(
                model = model, 
                false_positives = errors[1], 
                false_negatives = errors[2]
            ))
    }
}
errors = compute_errors(results = all_models[['ergm_model']], correct_results = gravity_results)
results = results %>%
    dplyr::bind_rows(data.frame(
        model = 'ergm_model',
        false_positives = errors[1], 
        false_negatives = errors[2]
    ))

In [203]:
results = results %>% 
    dplyr::group_by(model) %>%
    summarise(
        false_positives = mean(false_positives), 
        false_negatives = mean(false_negatives)
    )

In [209]:
merge(all_models[['general_models']][[4]], gravity_results, by = 'effect')

effect,p_value.x,sign_level.x,p_value.y,sign_level.y
<chr>,<dbl>,"<chr[,1]>",<dbl>,"<chr[,1]>"
absdiff-agriculture_forestry_fishing_of_gdp,0.020,*,1.898350e-04,***
absdiff-foreign_direct_investment_inflows,0.001,**,1.194428e-20,***
absdiff-gdp_growth,0.037,*,5.738196e-03,**
absdiff-gdp_us_dollar,0.000,***,2.058096e-30,***
absdiff-industry_of_gdp,0.200,,3.488696e-10,***
absdiff-inflation_rate,0.443,,4.634606e-01,
absdiff-merchandise_of_gdp,0.360,,9.939873e-01,
absdiff-net_barter_of_trade,0.159,,3.440619e-06,***
nodecov-agriculture_forestry_fishing_of_gdp,0.417,,1.349771e-03,**


In [206]:
results

model,false_positives,false_negatives
<chr>,<dbl>,<dbl>
ergm_model,5,2.0
general_models,1,6.0
residual_models,0,2.8


In [231]:
gravity_results

effect,p_value,sign_level
<chr>,<dbl>,"<chr[,1]>"
intercept,5.359752e-111,***
nodecov-industry_of_gdp,1.145209e-04,***
nodecov-merchandise_of_gdp,2.540912e-02,*
nodecov-agriculture_forestry_fishing_of_gdp,1.349771e-03,**
nodecov-gdp_growth,4.402026e-01,
nodecov-gdp_us_dollar,5.779273e-01,
nodecov-foreign_direct_investment_inflows,2.788586e-25,***
nodecov-inflation_rate,1.076154e-01,
nodecov-net_barter_of_trade,3.887822e-09,***


---